In [ ]:
from functools import partial
import numpy as np
import pandas as pd
from hyperopt import hp, fmin, tpe, Trials
from sklearn import metrics
from sklearn import model_selection
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('../input/mobile-price-classification/train.csv')

X = df.drop('price_range', axis=1).values
y = df['price_range'].values

In [ ]:
def optimize(params, x, y):
    model = make_pipeline(StandardScaler(), SVC(**params, probability=True))
    kf = model_selection.StratifiedKFold(n_splits=5)
    accuracies = []
    for idx in kf.split(X=x, y=y):
        train_idx, val_idx = idx[0], idx[1]
        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xval = x[val_idx]
        yval = y[val_idx]

        model.fit(xtrain, ytrain)
        preds = model.predict(xval)

        fold_acc = metrics.accuracy_score(yval, preds)
        accuracies.append(fold_acc)

    return -1.0 * np.mean(accuracies) # multiplied by -1 because the function finds the minimum.

In [ ]:
param_space = {
    'kernel': hp.choice('kernel', ['linear', 'poly', 'rbf', 'sigmoid']), # categorical
    'C': hp.uniform('C', 0.01, 100), # real
}  # hear order is important. If you have some kind of preprocessing you can add it here,
# but it must be passed to the function with the same sequence

optimization_function = partial(optimize, x=X, y=y)

trials = Trials()

result = fmin(
    fn=optimization_function,
    space=param_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials,
)

print(result)